In [26]:
# FOR SPY not BIOTECH

# # %%
# import os
# import openpyxl
# import pandas as pd
# from datetime import datetime, timedelta
# import wrds
# from sec_cik_mapper import StockMapper

# def install_packages():
#     """Install necessary packages."""
#     os.system('pip install wrds sec-cik-mapper openpyxl')

# def connect_to_wrds():
#     """Establish a connection to WRDS."""
#     try:
#         return wrds.Connection()
#     except Exception as e:
#         print(f"Error connecting to WRDS: {e}")
#         raise

# def configure_pgpass():
#     """Create and configure .pgpass file for secure connection."""
#     db.create_pgpass_file()
#     os.chmod(os.path.expanduser("~/.pgpass"), 0o600)

# def fetch_option_data(db, year, start_date, end_date):
#     """Fetch options data from WRDS."""
#     table = f'opprcd{year}'
#     query = f"""
#         SELECT date, exdate, cp_flag, strike_price, best_bid, best_offer,
#               impl_volatility, delta, gamma, vega, theta, optionid
#         FROM optionm.{table}
#         WHERE secid = '108105' -- This is the secid for SPX index options
#         AND date BETWEEN '{start_date}' AND '{end_date}'  
#         AND exdate < '{end_date}'::date + interval '30 days'  
#         ORDER BY date DESC  
#         LIMIT 100;
#     """
#     try:
#         return db.raw_sql(query)
#     except Exception as e:
#         print(f"Error fetching option data: {e}")
#         raise

# def load_excel_tickers(workbook_path, ticker_column_index):
#     """Load tickers from an Excel file."""
#     try:
#         workbook = openpyxl.load_workbook(workbook_path)
#         sheet = workbook.active
#         tickers = [
#             row[0] for row in sheet.iter_rows(min_row=2, min_col=ticker_column_index + 1, max_col=ticker_column_index + 1, values_only=True)
#             if row[0]
#         ]
#         return tickers
#     except Exception as e:
#         print(f"Error loading Excel file: {e}")
#         raise

# def map_tickers_to_ciks(tickers):
#     """Map tickers to their corresponding CIKs."""
#     mapper = StockMapper()
#     mapping = {}
#     unmapped = []

#     for ticker in tickers:
#         cik = mapper.ticker_to_cik.get(ticker)
#         if cik:
#             mapping[ticker] = cik
#         else:
#             print(f"No CIK found for ticker: {ticker}")
#             unmapped.append(ticker)

#     return mapping, unmapped

# def save_mapping_to_excel(mapping, unmapped, output_path):
#     """Save ticker to CIK mapping and unmapped tickers to a new Excel file."""
#     workbook = openpyxl.Workbook()
    
#     # Create sheet for mapped tickers
#     output_sheet = workbook.active
#     output_sheet.title = 'Ticker to CIK Mapping'
#     output_sheet.append(['Ticker', 'CIK'])

#     for ticker, cik in mapping.items():
#         output_sheet.append([ticker, cik])

#     # Create sheet for unmapped tickers
#     if unmapped:
#         unmapped_sheet = workbook.create_sheet(title='Unmapped Tickers')
#         unmapped_sheet.append(['Ticker'])
#         for ticker in unmapped:
#             unmapped_sheet.append([ticker])

#     workbook.save(output_path)
#     print(f"Mapping saved to {output_path}")

# def main():
#     # Install necessary packages
#     install_packages()

#     # Connect to WRDS
#     db = connect_to_wrds()

#     # Configure .pgpass file
#     configure_pgpass()

#     # Fetch options data
#     year = '2023'
#     start_date = f'{year}-07-10'
#     end_date = f'{year}-07-12'
    
#     option_data = fetch_option_data(db, year, start_date, end_date)
#     print(option_data.head())

#     # Load tickers from Excel
#     workbook_path = "/Users/mukeshwaranbaskaran/Downloads/github_AFP/Complete-List-of-Biotech-Stocks-Listed-on-NASDAQ-Jan-1-24.xlsx"
#     ticker_column_index = 2  # Adjust as needed

#     tickers = load_excel_tickers(workbook_path, ticker_column_index)

#     # Map tickers to CIKs
#     mapping, unmapped = map_tickers_to_ciks(tickers)

#     # Save mapping to a new Excel file
#     output_workbook_path = "/Users/mukeshwaranbaskaran/Downloads/github_AFP/Biotech_Stocks_Mapping.xlsx"
#     save_mapping_to_excel(mapping, unmapped, output_workbook_path)

#     # Close the WRDS connection
#     db.close()

# if __name__ == "__main__":
#     main()

DEPRECATION: Loading egg at /Users/mukeshwaranbaskaran/anaconda3/envs/hw2/lib/python3.12/site-packages/pandas-2.2.2-py3.12-macosx-11.0-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/mukeshwaranbaskaran/anaconda3/envs/hw2/lib/python3.12/site-packages/tzdata-2024.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/mukeshwaranbaskaran/anaconda3/envs/hw2/lib/python3.12/site-packages/pyportfolioopt-1.5.5-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loadin


[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
         date      exdate cp_flag  strike_price  best_bid  best_offer  \
0  2023-07-12  2023-08-10       P     5600000.0    1102.4      1111.1   
1  2023-07-12  2023-08-10       P     5400000.0     903.3       911.9   
2  2023-07-12  2023-08-10       P     5200000.0     704.2       712.8   
3  2023-07-12  2023-08-10       P     5000000.0     505.1       513.7   
4  2023-07-12  2023-08-10       P     4900000.0     405.6       414.3   

   impl_volatility     delta     gamma      vega     theta     optionid  
0         0.272328 -0.997707  0.000020   8.86615  252.0509  155582387.0  
1         0.238840 -0.996587  0.000034  12.77760  238.1409  155582386.0  
2         0.202422 -0.994525  0.000061  19.59587  222.2850  155582385.0  
3         0.158888 -0.991382  0.000116  29.34504  207.5751  155582384.0 

In [4]:
# %%
import os
import openpyxl
import pandas as pd
from datetime import datetime, timedelta
import wrds
from sec_cik_mapper import StockMapper

def install_packages():
    """Install necessary packages."""
    os.system('pip install wrds sec-cik-mapper openpyxl')

def connect_to_wrds():
    """Establish a connection to WRDS."""
    try:
        return wrds.Connection()
    except Exception as e:
        print(f"Error connecting to WRDS: {e}")
        raise

def configure_pgpass(db):
    """Create and configure .pgpass file for secure connection."""
    db.create_pgpass_file()
    os.chmod(os.path.expanduser("~/.pgpass"), 0o600)

def fetch_option_data(db, year, start_date, end_date, secid):
    """Fetch options data from WRDS."""
    table = f'opprcd{year}'
    query = f"""
        SELECT date, exdate, cp_flag, strike_price, best_bid, best_offer,
              impl_volatility, delta, gamma, vega, theta, optionid
        FROM optionm.{table}
        WHERE secid = '{secid}'
        AND date BETWEEN '{start_date}' AND '{end_date}'  
        AND exdate < '{end_date}'::date + interval '30 days'  
        ORDER BY date DESC  
        LIMIT 100;
    """
    try:
        return db.raw_sql(query)
    except Exception as e:
        print(f"Error fetching option data: {e}")
        raise

def map_ticker_to_cik(ticker):
    """Map a ticker to its corresponding CIK."""
    mapper = StockMapper()
    cik = mapper.ticker_to_cik.get(ticker)
    if cik:
        return cik
    else:
        print(f"No CIK found for ticker: {ticker}")
        return None

# def main():
#     # Install necessary packages
#     install_packages()

#     # Connect to WRDS
#     db = connect_to_wrds()

#     # Configure .pgpass file
#     configure_pgpass(db)

#     # Fetch options data
#     year = '2023'
#     start_date = f'{year}-07-10'
#     end_date = f'{year}-07-12'
    
#     ticker = 'TXG'
#     cik = map_ticker_to_cik(ticker)
#     print(cik)
    
#     if cik:
#         secid = f'{cik:010d}'  # Convert CIK to 10-digit secid format
#         print(secid)
#         option_data = fetch_option_data(db, year, start_date, end_date, secid)
#         print(option_data.head())
#     else:
#         print(f"Unable to fetch option data for ticker: {ticker}")

#     # Close the WRDS connection
#     db.close()

def main():
    # Install necessary packages
    install_packages()

    # Connect to WRDS
    db = connect_to_wrds()

    # Configure .pgpass file
    configure_pgpass(db)

    # Fetch options data
    year = '2023'
    start_date = f'{year}-07-10'
    end_date = f'{year}-07-12'
    
    ticker = 'TXG'
    cik = map_ticker_to_cik(ticker)
    print(cik)
    if cik:
        secid = cik.zfill(10)  # Ensure CIK is a 10-digit string
        print(secid)
        option_data = fetch_option_data(db, year, start_date, end_date, secid)
        print(option_data.head())
    else:
        print(f"Unable to fetch option data for ticker: {ticker}")

    # Close the WRDS connection
    db.close()


if __name__ == "__main__":
    main()

DEPRECATION: Loading egg at /Users/mukeshwaranbaskaran/anaconda3/envs/hw2/lib/python3.12/site-packages/pandas-2.2.2-py3.12-macosx-11.0-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/mukeshwaranbaskaran/anaconda3/envs/hw2/lib/python3.12/site-packages/tzdata-2024.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/mukeshwaranbaskaran/anaconda3/envs/hw2/lib/python3.12/site-packages/pyportfolioopt-1.5.5-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loadin


[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
0001770787
0001770787
Empty DataFrame
Columns: [date, exdate, cp_flag, strike_price, best_bid, best_offer, impl_volatility, delta, gamma, vega, theta, optionid]
Index: []


# DATABENTO


In [27]:
#!pip install -U databento

DEPRECATION: Loading egg at /Users/mukeshwaranbaskaran/anaconda3/envs/hw2/lib/python3.12/site-packages/pandas-2.2.2-py3.12-macosx-11.0-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/mukeshwaranbaskaran/anaconda3/envs/hw2/lib/python3.12/site-packages/tzdata-2024.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/mukeshwaranbaskaran/anaconda3/envs/hw2/lib/python3.12/site-packages/pyportfolioopt-1.5.5-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loadin

In [36]:
import databento as db
from datetime import datetime, timezone

# Initialize the client with your API key
client = db.Historical("API")

# Set the start date
start_date = "2024-10-01T00:00:00"

# Get the latest available date for the dataset
metadata = client.metadata.get_dataset_range(dataset="XNAS.ITCH")
end_date = "2024-10-10T00:00:00"  # metadata.end.isoformat()

# Retrieve equity data for TXG
equity_data = client.timeseries.get_range(
    dataset="XNAS.ITCH",  # NASDAQ ITCH dataset
    symbols="TXG",
    schema="trades",  # You can change this to "mbp-1" for top-of-book quotes
    start=start_date,
    end=end_date
)

# Convert equity data to a pandas DataFrame
equity_df = equity_data.to_df()
print("Equity Data:")
print(equity_df.head())
print(f"Total equity records: {len(equity_df)}")

Equity Data:
                                                               ts_event  \
ts_recv                                                                   
2024-10-01 13:30:00.920123878+00:00 2024-10-01 13:30:00.919938292+00:00   
2024-10-01 13:30:27.089093333+00:00 2024-10-01 13:30:27.088926601+00:00   
2024-10-01 13:31:00.422749715+00:00 2024-10-01 13:31:00.422583326+00:00   
2024-10-01 13:31:04.702191383+00:00 2024-10-01 13:31:04.702024283+00:00   
2024-10-01 13:31:04.702299044+00:00 2024-10-01 13:31:04.702129229+00:00   

                                     rtype  publisher_id  instrument_id  \
ts_recv                                                                   
2024-10-01 13:30:00.920123878+00:00      0             2          16371   
2024-10-01 13:30:27.089093333+00:00      0             2          16371   
2024-10-01 13:31:00.422749715+00:00      0             2          16371   
2024-10-01 13:31:04.702191383+00:00      0             2          16371   
2024-10-01 

In [43]:
from databento import Historical, HistoricalBuilder, Symbology
from datetime import datetime

# Initialize the Historical client
historical = Historical("API")

# Set the start and end dates
start_date = "2024-10-01"
end_date = "2024-10-10"

# Resolve the symbol to get all AAPL options
symbols = historical.metadata.resolve_symbols(
    ["AAPL"],
    dataset="OPRA.PILLAR",
    stype_in=Symbology.TICKER,
    stype_out=Symbology.INSTRUMENT_ID,
    date=datetime.strptime(start_date, "%Y-%m-%d")
)

# Create a list to store all AAPL option instrument IDs
aapl_options = []

for symbol, instrument_ids in symbols.items():
    aapl_options.extend(instrument_ids)

# Build the request for AAPL options data
request = (
    HistoricalBuilder()
    .dataset("OPRA.PILLAR")
    .symbols(aapl_options)
    .start(start_date)
    .end(end_date)
    .schema("mbo")  # You can change this to other schemas like "mbp" or "ohlcv" if needed
)

# Get the data
data = historical.timeseries.get_range(request)

# Process the data
for record in data:
    # Process each record as needed
    print(record)

ImportError: cannot import name 'HistoricalBuilder' from 'databento' (/Users/mukeshwaranbaskaran/anaconda3/envs/HW2/lib/python3.12/site-packages/databento/__init__.py)